# **PREDICCIONES PARA TODA LA POBLACION SIN AGREGACION**

In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [21]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 1
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_PREDICITIONS = True

In [3]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Individual/"

In [16]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

NUMBER_OF_PARTICIPANTS = dataX.shape[0]
print('\033[1m' + "\t SHAPES" + '\033[0m')
print(dataX.shape)
print(dataY.shape)

	 SHAPES
(100, 36355, 1440)
(100, 36355, 24)


In [17]:
dataX = dataX.transpose(1,0,2)
dataY = dataY.transpose(1,0,2)
print(dataX.shape)
print(dataY.shape)

(36355, 100, 1440)
(36355, 100, 24)


In [18]:
#We split a test set for testing
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=TEST_SIZE,shuffle=False)
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)
VALIDATION_INDEX = int(len(X_train)*VALIDATION_SIZE)
print("Examples for validation: \n", VALIDATION_INDEX)

Examples for training
 X: (29084, 100, 1440) y: (29084, 100, 24)
Examples for test
 X: (7271, 100, 1440) y: (7271, 100, 24)
Examples for validation: 
 4362


In [7]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[2]
WINDOW_SIZE = 1440

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(NUMBER_OF_PARTICIPANTS ,WINDOW_SIZE))
# x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128,return_sequences=True, activation="relu")(inputs) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

Model: "model_5_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 1440)]        0         
                                                                 
 lstm (LSTM)                 (None, 10, 128)           803328    
                                                                 
 dense (Dense)               (None, 10, 24)            3096      
                                                                 
Total params: 806,424
Trainable params: 806,424
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [27]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train[:-VALIDATION_INDEX],
            y_train[:-VALIDATION_INDEX],
            epochs=10,
            verbose=1,
            batch_size=128,
            validation_data=(X_train[-VALIDATION_INDEX:], y_train[-VALIDATION_INDEX:]))

Epoch 1/10
194/194 [==============================] - 83s 426ms/step - loss: 9477.7480 - mae: 9477.7500 - val_loss: 3865.3984 - val_mae: 3865.3982
Epoch 2/10
194/194 [==============================] - 90s 466ms/step - loss: 2931.2793 - mae: 2931.2781 - val_loss: 4489.7031 - val_mae: 4489.7026
Epoch 3/10
194/194 [==============================] - 86s 441ms/step - loss: 972156.3125 - mae: 972156.6875 - val_loss: 24017.9883 - val_mae: 24017.9902
Epoch 4/10
194/194 [==============================] - 104s 537ms/step - loss: 23457.4961 - mae: 23457.4961 - val_loss: 25029.5449 - val_mae: 25029.5371
Epoch 5/10
194/194 [==============================] - 101s 523ms/step - loss: 18735.4883 - mae: 18735.4824 - val_loss: 13772.0938 - val_mae: 13772.0938
Epoch 6/10
194/194 [==============================] - 97s 502ms/step - loss: 14002.4736 - mae: 14002.4795 - val_loss: 12473.9775 - val_mae: 12473.9785
Epoch 7/10
194/194 [==============================] - 91s 470ms/step - loss: 10568.6602 - mae: 105

In [28]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)
print(predictions[0])
if predictions.shape[1] == NUMBER_OF_PARTICIPANTS:
    predictions = np.sum(predictions,axis=1)
if y_test.shape[1] == NUMBER_OF_PARTICIPANTS:
    y_test = np.sum(y_test,axis=1)
print(y_test.shape)
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')

if COMPUTED_OPTION < 2:
    y_test_suma = np.sum(y_test,axis=1)
    predictions_suma = np.sum(predictions,axis=1)
    print(y_test_suma.shape)
    print(predictions_suma.shape)
    print('\033[1m' + "MSE: " + str(mean_squared_error(y_test_suma,predictions_suma)) + '\033[0m')
    print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test_suma,predictions_suma)) + '\033[0m')

228/228 [==============================] - 35s 155ms/step
tf.Tensor(
[[ 6.46476212e+01  6.57639847e+01  5.91623039e+01 ...  9.34137344e+01
   1.22830513e+02  7.29907303e+01]
 [ 4.00990334e+01  8.02103271e+01  8.31270981e+01 ...  1.27186646e+02
   9.44543152e+01  7.64020691e+01]
 [ 4.92350502e+01  9.49531097e+01  8.59246216e+01 ...  1.54925995e+02
   8.60619583e+01  9.16521988e+01]
 ...
 [ 1.58832949e+04  4.17788555e+04 -8.63707656e+04 ...  9.38564297e+04
  -2.31619082e+04  3.76382383e+04]
 [ 1.77194062e+04  4.69380781e+04 -9.77147422e+04 ...  1.05869805e+05
  -2.63549082e+04  4.24164883e+04]
 [ 1.97689121e+04  5.27386133e+04 -1.10548023e+05 ...  1.19413555e+05
  -2.99879082e+04  4.78052383e+04]], shape=(100, 24), dtype=float32)
(7271, 24)
MSE: 669335700000.0
MAE: 702147.56
(7271,)
(7271,)
MSE: 1202587200000.0
MAE: 878570.5


In [24]:
print(y_test[0])

[8169.599  7971.8545 7955.015  7980.65   7969.112  7950.568  7955.8857
 8110.5205 8184.1626 8522.484  8572.007  8644.12   8945.688  9122.947
 9264.303  8918.534  8492.656  8340.331  8982.699  8947.854  8887.069
 8991.166  8655.424  8382.224 ]


In [29]:
if COMPUTED_OPTION < 2:
    plot_predictions_vs_real(predictions[0],y_test[0])
    plot_predictions_vs_real(predictions_suma[:100],y_test_suma[:100])
else:
    plot_predictions_vs_real(predictions_suma[:100],y_test_suma[:100])

<Figure size 640x480 with 0 Axes>